In [22]:
import pandas as pd

data = pd.read_csv('./data/abcnews-date-text.csv',error_bad_lines=False)
data_text = data[['headline_text']]
data_text['index']=data_text.index
documents=data_text

In [23]:
print(len(documents))

1186018


In [26]:
print(documents[:5])

                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

unable to import 'smart_open.gcs', disabling that module
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mushirih\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def lemmatize_stemming(text):
    stemmer= SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS  and len(token)>3:
            result.append(lemmatize_stemming(token))
            
    return result

In [7]:
print(documents[documents['index']==4310])

                                          headline_text  index
4310  ratepayers group wants compulsory local govt v...   4310


In [8]:
doc_sample = documents[documents['index']==4310].values[0][0]

print('Original document:')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print (words)

print('\n\n tokenized and lemmatized doc:')
print(preprocess(doc_sample))

Original document:
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized doc:
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [9]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [10]:
# dictionary = gensim.corpora.Dictionary(processed_docs)
# count = 0

# for i, j in dictionary.iteritems():
#     for a, b in dictionary.iteritems():
#         if j == b:
#             count+=1
#             print(i,j, count)
        
#     count=0
   
# #     count+=1
# #     if count>10:
# #         break

In [11]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count=0

for k, v in dictionary.iteritems():
    print(k,v)
    count +=1
    if count>10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [12]:
dictionary.filter_extremes(no_below=15, no_above=.5, keep_n=100000)

In [13]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

In [14]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [15]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print('Word {} (\"{}\") appears {} times.'.format(bow_doc_4310[i][0],
                                                     dictionary[bow_doc_4310[i][0]],
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 times.
Word 240 ("group") appears 1 times.
Word 292 ("vote") appears 1 times.
Word 589 ("local") appears 1 times.
Word 838 ("want") appears 1 times.
Word 3567 ("compulsori") appears 1 times.
Word 3568 ("ratepay") appears 1 times.


In [16]:
from gensim import corpora, models
from pprint import pprint

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


In [17]:
lda = gensim.models.LdaMulticore(bow_corpus, num_topics=10,id2word=dictionary, passes=2, workers=2)

In [18]:
for idx, topic in lda.print_topics(-1):
    print('Topic: ',idx, ' Words: ', topic)

Topic:  0  Words:  0.022*"south" + 0.022*"hous" + 0.019*"north" + 0.016*"miss" + 0.015*"bushfir" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"coast" + 0.010*"investig"
Topic:  1  Words:  0.032*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.021*"dead" + 0.020*"polic" + 0.020*"attack" + 0.014*"offic" + 0.013*"assault" + 0.011*"michael" + 0.011*"bank"
Topic:  2  Words:  0.056*"australia" + 0.046*"australian" + 0.026*"world" + 0.017*"test" + 0.013*"win" + 0.011*"final" + 0.011*"farm" + 0.011*"open" + 0.010*"return" + 0.010*"canberra"
Topic:  3  Words:  0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder" + 0.020*"crash" + 0.020*"woman" + 0.017*"face" + 0.017*"alleg" + 0.013*"jail"
Topic:  4  Words:  0.019*"chang" + 0.018*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"concern" + 0.012*"climat" + 0.011*"flood" + 0.011*"fear" + 0.010*"save"
Topic:  5  Words:  0.021*"market" + 0.019*"news" + 0.018*"women" + 0.017*"live" + 0.016*"tasmania

In [19]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word= dictionary, passes =2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: ',idx, ' Word: ', topic)

Topic:  0  Word:  0.009*"stori" + 0.008*"commiss" + 0.008*"farm" + 0.008*"royal" + 0.006*"morrison" + 0.006*"govern" + 0.005*"financ" + 0.005*"andrew" + 0.005*"farmer" + 0.005*"care"
Topic:  1  Word:  0.021*"countri" + 0.015*"hour" + 0.007*"bushfir" + 0.006*"jam" + 0.006*"burn" + 0.006*"celebr" + 0.006*"april" + 0.006*"univers" + 0.005*"road" + 0.005*"money"
Topic:  2  Word:  0.017*"interview" + 0.009*"wednesday" + 0.009*"michael" + 0.008*"sport" + 0.007*"extend" + 0.007*"explain" + 0.007*"octob" + 0.007*"june" + 0.006*"mental" + 0.006*"social"
Topic:  3  Word:  0.011*"live" + 0.009*"weather" + 0.009*"friday" + 0.007*"juli" + 0.007*"energi" + 0.006*"august" + 0.006*"cattl" + 0.005*"march" + 0.005*"know" + 0.005*"blog"
Topic:  4  Word:  0.028*"trump" + 0.015*"donald" + 0.011*"australia" + 0.008*"cricket" + 0.007*"christma" + 0.006*"say" + 0.006*"outback" + 0.006*"novemb" + 0.005*"india" + 0.005*"asylum"
Topic:  5  Word:  0.022*"news" + 0.017*"rural" + 0.010*"street" + 0.009*"nation" + 0

In [20]:
for index, score in sorted(lda[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda.print_topic(index, 10)))




Score: 0.4510972797870636	 
Topic: 0.031*"govern" + 0.020*"warn" + 0.019*"feder" + 0.015*"countri" + 0.015*"fund" + 0.014*"claim" + 0.014*"life" + 0.013*"say" + 0.012*"health" + 0.011*"stori"

Score: 0.4488705098628998	 
Topic: 0.019*"donald" + 0.014*"nation" + 0.014*"farmer" + 0.013*"rural" + 0.013*"time" + 0.012*"council" + 0.012*"indigen" + 0.012*"school" + 0.011*"plan" + 0.011*"commiss"

Score: 0.01250685565173626	 
Topic: 0.036*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.012*"labor" + 0.012*"liber" + 0.011*"morrison" + 0.011*"leader" + 0.011*"parti" + 0.010*"campaign"

Score: 0.012503929436206818	 
Topic: 0.019*"chang" + 0.018*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"concern" + 0.012*"climat" + 0.011*"flood" + 0.011*"fear" + 0.010*"save"

Score: 0.012503573670983315	 
Topic: 0.022*"south" + 0.022*"hous" + 0.019*"north" + 0.016*"miss" + 0.015*"bushfir" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"coast" + 0.010*"investig"


In [21]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda[bow_vector], key =lambda tup: -1*tup[1]):
    print('Score: {} \t Topic: {}'.format(score,lda.print_topic(index,5)))

Score: 0.47889599204063416 	 Topic: 0.021*"market" + 0.019*"news" + 0.018*"women" + 0.017*"live" + 0.016*"tasmania"
Score: 0.22141841053962708 	 Topic: 0.036*"elect" + 0.018*"water" + 0.018*"state" + 0.016*"tasmanian" + 0.012*"labor"
Score: 0.18294937908649445 	 Topic: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder"
Score: 0.01667722873389721 	 Topic: 0.019*"chang" + 0.018*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker"
Score: 0.016677187755703926 	 Topic: 0.031*"govern" + 0.020*"warn" + 0.019*"feder" + 0.015*"countri" + 0.015*"fund"
Score: 0.01667635329067707 	 Topic: 0.022*"south" + 0.022*"hous" + 0.019*"north" + 0.016*"miss" + 0.015*"bushfir"
Score: 0.01667635329067707 	 Topic: 0.032*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.021*"dead" + 0.020*"polic"
Score: 0.01667635329067707 	 Topic: 0.056*"australia" + 0.046*"australian" + 0.026*"world" + 0.017*"test" + 0.013*"win"
Score: 0.01667635329067707 	 Topic: 0.019*"donald" + 0.014*"nation" + 0.01